In [3]:
from syncode import Syncode
import warnings
warnings.filterwarnings('ignore')

model_name = "/data/share/models/hugging_face/Llama-7b"
grammar_file = open("invariants.lark", "r") 
grammar = grammar_file.read()
grammar_file.close()

# Load the Syncode augmented model
syn_llm = Syncode(model = model_name, mode='grammar_mask', grammar=grammar, parse_output_only=True, num_samples=15, log_level=2)

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.88it/s]
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [4]:
prompt = """You are a helpful AI software assistant that reasons about how code behaves. Given a program, you can find loop invariants, which can then be used to verify some property in the program. 
Frama-C is a software verification tool for C programs. The input to Frama-C is a C program file with ACSL (ANSI/ISO C Specification Language) annotations.
For the given program, find the necessary loop invariants of the while loop to help Frama-C verify the post-condition.

Instructions:
- Make a note of the pre-conditions or variable assignments in the program.
- Analyze the loop body and make a note of the loop condition. 
- Output loop invariants that are true 
(i) before the loop execution, 
(ii) in every iteration of the loop and 
(iii) after the loop termination, 
such that the loop invariants imply the post condition.
- If a loop invariant is a conjunction, split it into its parts.
- Output all the loop invariants in one code block. For example:
```
/*@ 
    loop invariant i1;
    loop invariant i2;
*/
```
Rules:
- **Do not use variables or functions that are not declared in the program.** 
- **Do not make any assumptions about functions whose definitions are not given.**
- **All undefined variables contain garbage values. Do not use variables that have garbage values.**
- **Do not use keywords that are not supported in ACSL annotations for loops.**
- **Variables that are not explicitly initialized, could have garbage values. Do not make any assumptions about such values.**
- **Do not use the \at(x, Pre) notation for any variable x.**
- **Do not use non-deterministic function calls.**

Consider the following C program:
```
{code}
```

You are allowed to use implication to take care of the conditional nature of the code. Use implication (==>) instead of using if-then.

For all variables, add conjunctions that bound the maximum and minimum values that they can take, if such bounds exist.

If a variable is always equal to or smaller or larger than another variable, add a conjunction for their relation.

If the assertion is guarded by a condition, use the guard condition in an implication.

If certain variables are non-deterministic at the beginning or end of the loop, use an implication to make the invariant trivially true at that location. 

Output the loop invariants for the loop in the program above. Let's think step by step.
"""

In [5]:
import json

In [6]:
benchmarks = []
for i in range(1, 26):
    with open(f"dataset/code2inv/{i}.c") as f:
        code = f.read()
        code = code.split("\n")[1:]
        code = "\n".join(code)
        benchmarks.append({"file": f"dataset/code2inv/{i}.c", "code": code})

In [17]:
benchmarks[0]

{'file': 'dataset/code2inv/1.c',
 'code': '#include <stdlib.h>\n#define assume(e) if(!(e)) exit(-1);\n\nint main() {\n  \n  int x;\n  int y;\n  \n  (x = 1);\n  (y = 0);\n  \n  while ((y < 100000)) {\n    {\n    (x  = (x + y));\n    (y  = (y + 1));\n    }\n\n  }\n  \n{;\n//@ assert( (x >= y) );\n}\n\n}'}

In [18]:
import os

if not os.path.exists("results"):
    os.makedirs("results")

In [9]:
for i, benchmark in enumerate(benchmarks):
    invariants = syn_llm.infer(prompt.format(code=benchmark["code"]), stop_words=None)
    with open(f"results/{i+1}.json", "w") as f:
        json.dump(invariants, f)

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.39 GiB. GPU 0 has a total capacity of 23.58 GiB of which 980.50 MiB is free. Including non-PyTorch memory, this process has 22.60 GiB memory in use. Of the allocated memory 19.05 GiB is allocated by PyTorch, and 3.31 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
bench